In [159]:
from newsgac import database
from newsgac import config
from newsgac.data_sources import DataSource
from newsgac.pipelines import Pipeline
from newsgac.nlp_tools.models.frog import Frog
from newsgac.nlp_tools.tasks import frog_process, frogclient
from pynlpl.clients.frogclient import FrogClient
import pandas
import nltk
nltk.download('punkt')
frogclient = FrogClient(
            config.frog_hostname,
            config.frog_port,
            returnall=True,
            timeout=1800.0,
        )

[nltk_data] Downloading package punkt to /home/tom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
[d.display_title for d in DataSource.objects.all()]

[u'NBGS Training', u'NBGS Testing', u'nbgs testing4training']

In [14]:
data_source = DataSource.objects.first()
print data_source.articles[0].raw_text

Dr.H.S . Visscher Filmkroniek FILM ZONDER NOODZAAK OF RAF FINEMENT „I TABU ” : SIGHT-SEEING VAN MENSELIJKE NARIGHEID `` T\E mogelijkheden van de „verborgen camera '' en „cinema vérité '' hebben een filmgenre in het leven geroepen waarvan het voortbestaan even onmiskenbaar als raadselachtig is : de exotische en erotische reportage- en compilatiefilm . Cameraploegen worden over de hele wereld uitgestuurd op zoek naar bizarre en morbide verschijnselen in de menselijke samenleving , en vervolgens wordt het materiaal ondergebracht in een „kijk-film '' , die onder allerlei eerlijkheids- en onthullingsleuzen op ongelimiteerde wijze de voyeursneigingen die de „candid camera '' eigen zgn , uitbuit . Het resultaat is in de meeste gevallen een kleurige sight-seeing van menselijke narigheid en buitenissigheid met een stevige dosis exotisch naakt . Blasetti — het genre is bij uitstek Italiaans — voerde ons langs allerlei soorten van liefde en sexualiteit „all over the world '' in zijn film „ik bemi

In [4]:
p = Pipeline.create()

In [5]:
p.lemmatization = False
p.sw_removal = False
p.quote_removal = True

In [6]:
p.nlp_tool = Frog.create()

In [7]:
skp = p.get_sk_pipeline()

In [8]:
skp.steps

[('CleanOCR', <newsgac.nlp_tools.transformers.CleanOCR at 0x7fabb8ec1850>),
 ('FeatureExtraction', FeatureUnion(n_jobs=None,
         transformer_list=[('BasicFeatures', <newsgac.nlp_tools.transformers.ExtractBasicFeatures object at 0x7fabb3457e50>), ('QuoteFeatures', <newsgac.nlp_tools.transformers.ExtractQuotes object at 0x7fabb34430d0>), ('SentimentFeatures', <newsgac.nlp_tools.transformers.ExtractSentimentFeatures object at 0x...88a90>), ('Frog', <newsgac.nlp_tools.models.frog.FrogFeatureExtractor object at 0x7fabb3288b10>)]))],
         transformer_weights=None)),
 ('RobustScaler',
  RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=False,
         with_scaling=True)),
 ('Classifier', SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel=u'linear',
    max_iter=-1, probability=True, random_state=42, shrinking=True,
    tol=0.001, verbose=False))]

In [9]:
skp.steps.pop()
skp.steps.pop()

('RobustScaler',
 RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=False,
        with_scaling=True))

In [10]:
skp.steps

[('CleanOCR', <newsgac.nlp_tools.transformers.CleanOCR at 0x7fabb8ec1850>),
 ('FeatureExtraction', FeatureUnion(n_jobs=None,
         transformer_list=[('BasicFeatures', <newsgac.nlp_tools.transformers.ExtractBasicFeatures object at 0x7fabb3457e50>), ('QuoteFeatures', <newsgac.nlp_tools.transformers.ExtractQuotes object at 0x7fabb34430d0>), ('SentimentFeatures', <newsgac.nlp_tools.transformers.ExtractSentimentFeatures object at 0x...88a90>), ('Frog', <newsgac.nlp_tools.models.frog.FrogFeatureExtractor object at 0x7fabb3288b10>)]))],
         transformer_weights=None))]

In [174]:
article_num = 5
cleaned_text = skp.steps[0][1].transform([data_source.articles[article_num].raw_text])[0]
print cleaned_text

Geen concessies in oudejaarsrede De Gaulle houdt vast aan eigen weg In 1965 definitief uit de NATO ? Van onze correspondent PARIJS , 2 jan. — In zijn ondcjaarsradio en televisierede heeft generaal dc Gaulle alle illusies , die hier en daar hebben postgevat over eenmeer tegemoetkomende houding van Frankrijk ten opzichte van zijn Europese partners en Atlantische bondgenoten , met een forse streek van dc kaart geveegd . Dc president heeft te verstaan gegeven , dat er van hem niet dc geringste concessie te verwachten is op het gebied van dc stipranationaliteit , dc integratie en wat hij voorde eerste keer het „atlantisme '' noemde . Daarentegen heeft hij even duidelijk te verstaan gegeven , dat Frankrijk op ieder gebied zijn eigen baas wil worden en blijven . In Parijs , waar zelfs dc regering dc rede van generaal dc Gaulle met verbazing heeft ontvangen , beschouwt men dc uitspraken van dc president van dc republiek als dc onvoorwaardelijke afwijzing van iedere vorm van multilaterale óf at

In [18]:
unquoted_text = skp.steps[1][1].transformer_list[3][1].steps[0][1].transform([cleaned_text])[0]
print unquoted_text

Dr.H.S . Visscher Filmkroniek FILM ZONDER NOODZAAK OF RAF FINEMENT: SIGHT-SEEING VAN MENSELIJKE NARIGHEID TE mogelijkheden van de en hebben een filmgenre in het leven geroepen waarvan het voortbestaan even onmiskenbaar als raadselachtig is : de exotische en erotische reportage- en compilatiefilm . Cameraploegen worden over de hele wereld uitgestuurd op zoek naar bizarre en morbide verschijnselen in de menselijke samenleving , en vervolgens wordt het materiaal ondergebracht in een , die onder allerlei eerlijkheids- en onthullingsleuzen op ongelimiteerde wijze de voyeursneigingen die de eigen zgn , uitbuit . Het resultaat is in de meeste gevallen een kleurige sight-seeing van menselijke narigheid en buitenissigheid met een stevige dosis exotisch naakt . Blasetti — het genre is bij uitstek Italiaans — voerde ons langs allerlei soorten van liefde en sexualiteit in zijn film , die overigens nog gelardeerd werd met show-fragmenten die niets met zijn , thema ' te maken hadden . Een andere fil

In [33]:
frog_step = skp.steps[1][1].transformer_list[3][1].steps[1][1]
pandas.DataFrame(frog_step.transform([unquoted_text]), columns=frog_step.get_feature_names())


,adjectives_perc,cogn_verbs_perc,intensifiers_perc,modal_adverbs_perc,modal_verbs_perc,named_entities_perc,number_perc,pronoun_1_perc,pronoun_2_perc,pronoun_3_perc,unique_named_entities
0,0.106285,0.003697,0.009242,0.001848,0.00647,0.039741,0.002773,0.008318,0.0,0.014787,0.604651


In [23]:
frog_step.get_feature_names()

['adjectives_perc',
 'cogn_verbs_perc',
 'intensifiers_perc',
 'modal_adverbs_perc',
 'modal_verbs_perc',
 'named_entities_perc',
 'number_perc',
 'pronoun_1_perc',
 'pronoun_2_perc',
 'pronoun_3_perc',
 'unique_named_entities']

In [25]:
frog_step.transform([unquoted_text])[0]

array([0.10628466, 0.00369686, 0.00924214, 0.00184843, 0.0064695 ,
       0.03974122, 0.00277264, 0.00831793, 0.        , 0.01478743,
       0.60465116])

In [56]:
115/1082.

0.10628465804066543

In [101]:
df = pandas.DataFrame(frog_process(unquoted_text))

In [170]:
l = df[df[4].map(lambda c: c[0:1]) == 'B'].groupby(1).size().reset_index().sort_values(1)

In [171]:
df[df[4].map(lambda c: c[0:1]) == 'B']

,0,1,2,3,4,5,6,7
0,Dr.H.S,Dr.H.S,[Dr.H.S],SPEC(deeleigen),B-PER,B-NP,,
2,Visscher_Filmkroniek,Visscher_Filmkroniek,[Visscher]_[Filmkroniek],SPEC(deeleigen)_SPEC(deeleigen),B-PER_I-PER,B-NP_I-NP,,
5,NOODZAAK,NOODZAAK,[NOODZAAK],SPEC(deeleigen),B-LOC,B-NP,,
7,RAF_FINEMENT,RAF_FINEMENT,[RAF]_[FINEMENT],SPEC(deeleigen)_SPEC(deeleigen),B-ORG_I-ORG,B-NP_I-NP,,
9,SIGHT-SEEING,sight-seeing,[sight-seeing],"N(soort,ev,basis,zijd,stan)",B-MISC,B-NP,,
109,Blasetti,Blasetti,[Blasetti],SPEC(deeleigen),B-PER,B-NP,,
115,Italiaans,Italiaans,[Italiaans],SPEC(deeleigen),B-LOC,I-NP,,
159,Europese,europees,[Europees][e],"ADJ(prenom,basis,met-e,stan)",B-LOC,I-NP,,
174,Jacopetti,Jacopetti,[Jacopetti],SPEC(deeleigen),B-PER,B-NP,,
207,Mondo_Cane,Mondo_Cane,[Mondo]_[Cane],SPEC(deeleigen)_SPEC(deeleigen),B-PER_I-PER,B-NP_I-NP,,


In [161]:
len(frogclient.process(cleaned_text))

1272

In [127]:
# Unique named entities
tokens = frog_process(unquoted_text)
named_entities = [t for t in tokens if t[4].startswith('B')]
unique_ne_strings = []
ne_strings = set([t[1].lower() for t in named_entities])
for ne_source in ne_strings:
    unique = True
    for ne_target in [n for n in ne_strings if n != ne_source]:
        if ne_target.find(ne_source) > -1:
            unique = False
            break
    if unique:
        unique_ne_strings.append(ne_source)


[u'blasetti',
 u'bombay',
 u'dr.h.s',
 u'europees',
 u'ganges',
 u'hiroshima',
 u'hollywoodtraditie',
 u'honoloeloe',
 u'i',
 u'india',
 u'italiaans',
 u'jacopetti',
 u'japan',
 u'japans',
 u'mille',
 u'mondo_cane',
 u'mondo_cane_ii',
 u'nederland',
 u'nederlands',
 u'noodzaak',
 u'peter_stuyvesantsigaretten',
 u'raf_finement',
 u'sight-seeing',
 u'tabu',
 u'visscher_filmkroniek',
 u'vlemmen',
 u'zweden',
 u'zweeds',
 u'zwitserland',
 u'zwitsers']

In [128]:
set(sorted(ne_strings)) - set(sorted(unique_ne_strings))

{u'i', u'japan', u'mondo_cane', u'nederland'}

In [129]:
skp.steps

[('CleanOCR', <newsgac.nlp_tools.transformers.CleanOCR at 0x7fabb8ec1850>),
 ('FeatureExtraction', FeatureUnion(n_jobs=None,
         transformer_list=[('BasicFeatures', <newsgac.nlp_tools.transformers.ExtractBasicFeatures object at 0x7fabb3457e50>), ('QuoteFeatures', <newsgac.nlp_tools.transformers.ExtractQuotes object at 0x7fabb34430d0>), ('SentimentFeatures', <newsgac.nlp_tools.transformers.ExtractSentimentFeatures object at 0x...88a90>), ('Frog', <newsgac.nlp_tools.models.frog.FrogFeatureExtractor object at 0x7fabb3288b10>)]))],
         transformer_weights=None))]

In [179]:
skp.steps[1][1].transformer_list[1][1].transform([cleaned_text])[0]


array([2.        , 0.07407407])

In [142]:
zip(basic_features.get_feature_names(), basic_features.transform([cleaned_text])[0])

[('question_marks_perc', 0.0008375209380234506),
 ('exclamation_marks_perc', 0.0),
 ('currency_symbols_perc', 0.0),
 ('digits_perc', 0.0016750418760469012),
 ('sentences', 32.0),
 ('avg_sentence_length', 37.3125)]

In [143]:
from nltk import word_tokenize

In [147]:
len(word_tokenize(cleaned_text))

1194

In [162]:
len(frogclient.process(cleaned_text))

1272

In [175]:
skp.steps[1][1].transformer_list[2][1].transform([cleaned_text])[0]

array([0.07121212, 0.40681818])

In [172]:
skp.steps[1][1].transformer_list[2][1].get_feature_names()

['polarity', 'subjectivity']

In [183]:
2. / 27

0.07407407407407407

In [184]:
from nltk import sent_tokenize

In [186]:
sent_tokenize('....a asd ads asd.. asdsadaasdlkj  gfgfdf fdgdf . sdgsd lsdfj with dr. bla die hospital.')

['....a asd ads asd.. asdsadaasdlkj  gfgfdf fdgdf .',
 'sdgsd lsdfj with dr. bla die hospital.']